In [1]:
!pip uninstall torchtext
!pip install torchtext    # we need torchtext>0.6

Uninstalling torchtext-0.3.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/test/common/*
    /usr/local/lib/python3.6/dist-packages/test/data/*
    /usr/local/lib/python3.6/dist-packages/torchtext-0.3.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchtext/*
Proceed (y/n)? y
  Successfully uninstalled torchtext-0.3.1
     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 1.1MB 6.9MB/s 


In [2]:
import torch
import spacy
# import torchtext.data.metrics
from torchtext.data.metrics import bleu_score
import sys


def translate_sentence(model, sentence, german, english, device, max_length=50):
    # print(sentence)

    # sys.exit()

    # Load german tokenizer
    spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence[1:]


def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import numpy as np
import spacy
import random

In [4]:
!python -m spacy download de

     |████████████████████████████████| 14.9MB 621kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=82b390cffd1fbba37e44fe824ff2875033e523a8826938b3b964b853520864b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-7wawin1z/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [5]:
spacy_ger = spacy.load('de')
spacy_eng = spacy.load('en')

In [6]:
def tokenizer_ger(text):
  return [tok.text for tok in spacy_ger.tokenizer(text)]

def tokenizer_eng(text):
  return [tok.text for tok in spacy_eng.tokenizer(text)]

In [7]:
german = Field(tokenize=tokenizer_ger, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenizer_eng, lower=True, init_token='<sos>', eos_token='<eos>')

In [14]:
train_data, valid_data, test_data = Multi30k.splits(
    exts=(".de", ".en"), fields=(german, english)
)

In [15]:
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

In [16]:
class Encoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(Encoder, self).__init__()
    self.dropout = nn.Dropout(p)
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.embedding = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

  def forward(self, x):
    # x shape: (seq_length, N) where N is batch size

    embedding = self.dropout(self.embedding(x))
    # embedding shape: (seq_length, N, embedding_size)

    outputs, (hidden, cell) = self.rnn(embedding)
    # outputs shape: (seq_length, N, hidden_size)

    return hidden, cell

class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p):
    super(Decoder, self).__init__()
    self.dropout = nn.Dropout(p)
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.embedding = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, hidden, cell):
  
    x = x.unsqueeze(0)

    embedding = self.dropout(self.embedding(x))
    # embedding shape: (1, N, embedding_size)

    outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
    # outputs shape: (1, N, hidden_size)

    predictions = self.fc(outputs)

    predictions = predictions.squeeze(0)

    return predictions, hidden, cell

In [17]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder):
    super(Seq2Seq, self).__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, source, target, teacher_force_ratio=0.5):
    batch_size = source.shape[1]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)

    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    hidden, cell = self.encoder(source)

    # Grab the first input to the Decoder which will be <SOS> token
    x = target[0]

    for t in range(1, target_len):
      # Use previous hidden, cell as context from encoder at start
      output, hidden, cell = self.decoder(x, hidden, cell)

      # Store next output prediction
      outputs[t] = output
      # Get the best word the Decoder predicted (index in the vocabulary)
      best_guess = output.argmax(1)
      x = target[t] if random.random() < teacher_force_ratio else best_guess

    return outputs

In [18]:
# Training hyperparameters
num_epochs = 20
learning_rate = 0.001
batch_size = 64

# Model hyperparameters
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size_encoder = len(german.vocab)
input_size_decoder = len(english.vocab)
output_size = len(english.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024  # Needs to be the same for both RNN's
num_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5

In [19]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,
)


In [20]:
encoder_net = Encoder(
    input_size_encoder, 
    encoder_embedding_size, 
    hidden_size, 
    num_layers, 
    enc_dropout
).to(device)

decoder_net = Decoder(
    input_size_decoder,
    decoder_embedding_size,
    hidden_size,
    output_size,
    num_layers,
    dec_dropout,
).to(device)

In [21]:
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


In [22]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

In [23]:
sentence = "ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen."

for epoch in range(num_epochs):
  print(f"[Epoch {epoch} / {num_epochs}]")

  checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
  save_checkpoint(checkpoint)

  model.eval()

  translated_sentence = translate_sentence(
      model, sentence, german, english, device, max_length=50
  )

  print(f"Translated example sentence: \n {translated_sentence}")

  model.train()

  for batch_idx, batch in enumerate(train_iterator):
    # Get input and targets and get to cuda
    inp_data = batch.src.to(device)
    target = batch.trg.to(device)

    # Forward prop
    output = model(inp_data, target)

    # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
    # doesn't take input in that form. For example if we have MNIST we want to have
    # output to be: (N, 10) and targets just (N). Here we can view it in a similar
    # way that we have output_words * batch_size that we want to send in into
    # our cost function, so we need to do some reshapin. While we're at it
    # Let's also remove the start token while we're at it
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    optimizer.zero_grad()
    loss = criterion(output, target)

    # Back prop
    loss.backward()

    # Clip to avoid exploding gradient issues, makes sure grads are
    # within a healthy range
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Gradient descent step
    optimizer.step()



score = bleu(test_data[1:100], model, german, english, device)
print(f"Bleu score {score*100:.2f}")

[Epoch 0 / 20]
=> Saving checkpoint
Translated example sentence: 
 ['giant', 'leading', 'oven', 'weapons', 'names', 'delight', 'delight', 'sculptor', 'wooded', 'wooded', 'flaming', 'grocery', 'grocery', 'electronics', 'presentation', 'follows', 'examining', 'examining', 'information', 'intersection', 'intersection', 'peaceful', 'singers', 'coconuts', 'pans', 'source', 'lively', 'lively', 'scythe', 'grocery', 'grocery', 'kayak', 'kayak', 'actions', 'festive', 'festive', '`', 'examining', 'examining', 'business', 'examining', 'high', 'examining', 'information', 'pulls', 'intersection', 'intersection', 'singers', 'outfit', 'outfit']
[Epoch 1 / 20]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'child', 'in', 'a', 'a', 'shirt', 'and', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<eos>']
[Epoch 2 / 20]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'worker', 'with', 'a', '<unk>', 'is', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<eos>']
[Epoch 3 / 20]
=> Saving checkpoint
Tra